In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_spectrogram_X_and_Y(X_vector_len=X_vector_len, take_log=True, split=0.85)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


# Train
* Hmm, sparse matrix doesnt work

In [3]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.test_spectrogram_conv import *

In [4]:
graph_name = 'TestSpectrogramNoConv'  # Specify graph class name

g_cnfg = Config('trial_graph')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[1].shape[1]
g_cnfg.n_hidden = 50
g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 0.7  # 1.0 means no decay

t_cnfg = Config('trial_run_01')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.dropout_keep_prob = 0.5
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0]/t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [5]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: trial_graph
X_img_h: 129
X_img_w: 71
Y_vector_len: 31
lr_decay_rate: 0.7
lr_decay_steps: 1,000,000
lr_initial: 0.1
n_hidden: 50

name: trial_run_01
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.5
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 50
max_step: 100,000,000
n_ave_ll_valid: 8
print_every: 367
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

TestSpectrogramNoConv_trial_graph_trial_run_01
Training starts @ 12/09/2017 12:12:55
Epoch 1 Step 1 Best model saved @ 12/09/2017 12:12:56 [Ave valid] 3.745
Epoch 1 Step 1 ends @ 12/09/2017 12:12:56 [Train] 3.239, 12.0% [Valid] 3.9% [Ave valid] 3.745
Epoch 1 Step 50 Best model saved @ 12/09/2017 12:12:58 [Ave valid] 3.671
Epoch 1 Step 367 ends @ 12/09/2017 12:13:10 [Train] 3.396, 6.8% [Valid] 4.1% [Ave valid] 3.674
Epoch 2 Step 400 Best model saved @ 12/09/2017 12:13:13 [Ave valid] 3.653
Epoch 2 Step 450 Best model saved @ 12/09/2017 12:13:15 [Ave valid] 3.520
Epoch 2 Step 550 Best mode

Epoch 48 Step 17,616 ends @ 12/09/2017 12:25:01 [Train] 3.372, 6.0% [Valid] 6.7% [Ave valid] 3.320
Epoch 49 Step 17,983 ends @ 12/09/2017 12:25:16 [Train] 3.231, 9.4% [Valid] 7.8% [Ave valid] 3.314
Epoch 50 Step 18,350 ends @ 12/09/2017 12:25:31 [Train] 3.195, 12.0% [Valid] 7.7% [Ave valid] 3.320
Epoch 51 Step 18,450 Best model saved @ 12/09/2017 12:25:36 [Ave valid] 3.301
Epoch 51 Step 18,717 ends @ 12/09/2017 12:25:46 [Train] 3.217, 6.8% [Valid] 6.3% [Ave valid] 3.308
Epoch 52 Step 19,084 ends @ 12/09/2017 12:26:01 [Train] 3.229, 6.0% [Valid] 7.0% [Ave valid] 3.316
Epoch 53 Step 19,451 ends @ 12/09/2017 12:26:16 [Train] 3.150, 9.4% [Valid] 7.1% [Ave valid] 3.306
Epoch 54 Step 19,818 ends @ 12/09/2017 12:26:31 [Train] 3.251, 11.1% [Valid] 7.4% [Ave valid] 3.371
Epoch 55 Step 20,185 ends @ 12/09/2017 12:26:46 [Train] 3.344, 6.0% [Valid] 5.1% [Ave valid] 3.334
Epoch 56 Step 20,552 ends @ 12/09/2017 12:27:02 [Train] 3.243, 8.5% [Valid] 6.5% [Ave valid] 3.328
Epoch 57 Step 20,919 ends @ 1

KeyboardInterrupt: 